
# 🎲 Indovinello di Shannon
## Binario vs k-ario con introduzione all'entropia

**Uso didattico**: prima si gioca (come nel gioco 1), poi alla fine si introduce l'**entropia** e si confronta il caso **uniforme** vs **non uniforme**.


## 0) Requisiti (eseguire una volta se necessario)

In [ ]:

# Se manca qualcosa, scommenta ed esegui:
# !pip install ipywidgets pandas matplotlib
# !jupyter nbextension enable --py widgetsnbextension


## 1) Import e stato di sessione

In [6]:

import math
import random
from dataclasses import dataclass, field
from typing import List, Optional
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np

SESSION_RESULTS = []  # risultati dei round giocati



## 2) Motore di gioco (uguale al gioco 1)
Gioca con domande **binarie** o **k-arie** e registra i tentativi. La teoria viene **dopo**.


In [7]:

@dataclass
class GameState:
    N: int = 64
    mode: str = "binary"  # "binary" or "kary"
    k: int = 4            # valido se mode == "kary"
    secret: int = field(default_factory=lambda: 1)
    candidates: List[int] = field(default_factory=list)
    attempts: int = 0
    finished: bool = False
    history: List[str] = field(default_factory=list)

    def reset(self, N: Optional[int] = None, mode: Optional[str] = None, k: Optional[int] = None, seed: Optional[int] = None):
        if N is not None:
            self.N = N
        if mode is not None:
            self.mode = mode
        if k is not None:
            self.k = k
        if seed is not None:
            random.seed(seed)
        self.secret = random.randint(1, self.N)
        self.candidates = list(range(1, self.N + 1))
        self.attempts = 0
        self.finished = False
        self.history = []
    
    def answer_binary_subset(self, subset: List[int]) -> bool:
        """Risponde se il segreto è nel sottoinsieme (SÌ/NO) e aggiorna i candidati."""
        if self.finished:
            return True
        self.attempts += 1
        yes = self.secret in subset
        before = len(self.candidates)
        if yes:
            self.candidates = sorted(list(set(self.candidates).intersection(subset)))
            self.history.append(f"Q{self.attempts} (bin): È nel sottoinsieme? → SÌ (cand: {before} → {len(self.candidates)})")
        else:
            self.candidates = sorted(list(set(self.candidates) - set(subset)))
            self.history.append(f"Q{self.attempts} (bin): È nel sottoinsieme? → NO (cand: {before} → {len(self.candidates)})")
        if len(self.candidates) == 1 and self.candidates[0] == self.secret:
            self.finished = True
        return yes
    
    def answer_kary_partition(self, parts: List[List[int]]) -> int:
        """Data una partizione in k sottoinsiemi, risponde con l'indice della parte che contiene il segreto e filtra i candidati."""
        if self.finished:
            return -1
        self.attempts += 1
        truth_index = -1
        for i, p in enumerate(parts):
            if self.secret in p:
                truth_index = i
                break
        before = len(self.candidates)
        if truth_index >= 0:
            self.candidates = sorted(list(set(self.candidates).intersection(parts[truth_index])))
        self.history.append(f"Q{self.attempts} (k={len(parts)}): in quale parte cade? → parte {truth_index+1 if truth_index>=0 else '?'} (cand: {before} → {len(self.candidates)})")
        if len(self.candidates) == 1 and self.candidates[0] == self.secret:
            self.finished = True
        return truth_index
    
    def guess(self, x: int) -> bool:
        """Tentativo finale: indovina il numero."""
        if self.finished:
            return True
        self.attempts += 1
        correct = (x == self.secret)
        self.history.append(f"Q{self.attempts} (guess): è {x}? → {'CORRETTO' if correct else 'sbagliato'}")
        if correct:
            self.finished = True
            self.candidates = [self.secret]
        else:
            if x in self.candidates:
                self.candidates.remove(x)
        return correct

def record_result(state: GameState, label: str = ""):
    SESSION_RESULTS.append({
        "mode": state.mode if label == "" else f"{state.mode}:{label}",
        "k": state.k if state.mode == "kary" else 2,
        "N": state.N,
        "attempts": state.attempts,
        "found": state.finished,
        "secret": state.secret,
        "history": list(state.history)
    })



## 3) Interfaccia di gioco (giocare **prima** della teoria)
1. Scegli **N**, **modalità** e (se k-aria) il valore di **k**.
2. **Nuovo round** → poni domande binarie o k-arie.
3. **Indovina** quando vuoi e **registra** il round.


In [ ]:

# Widgets
N_widget = widgets.BoundedIntText(value=64, min=4, max=100000, step=1, description="N (1..N):")
mode_widget = widgets.Dropdown(options=[("Binario (sì/no)", "binary"), ("Non binario (k-ario)", "kary")], value="binary", description="Modalità:")
k_widget = widgets.BoundedIntText(value=4, min=3, max=10, step=1, description="k (≥3):")
seed_widget = widgets.IntText(value=0, description="Seed (opz):")
new_round_btn = widgets.Button(description="🎯 Nuovo round", button_style="primary")
status_out = widgets.Output()

# Binary widgets
bin_range_start = widgets.BoundedIntText(value=1, min=1, max=64, description="Inizio:")
bin_range_end = widgets.BoundedIntText(value=32, min=1, max=64, description="Fine:")
ask_bin_btn = widgets.Button(description="❓ È nel range?", button_style="")

# K-ary widgets
kary_partition_btn = widgets.Button(description="⚖️ Partizione bilanciata", button_style="")
kary_apply_btn = widgets.Button(description="❓ In quale parte?", button_style="")
k_parts_boxes = [widgets.Textarea(value="", description=f"Parte {i+1}", layout=widgets.Layout(width="100%", height="60px")) for i in range(10)]
kary_info_out = widgets.Output()

# Guess & record
guess_input = widgets.BoundedIntText(value=1, min=1, max=64, description="Prova:")
guess_btn = widgets.Button(description="✅ Indovina", button_style="success")
record_btn = widgets.Button(description="💾 Registra round", button_style="warning")
results_out = widgets.Output()

# History/candidates
history_out = widgets.Output()
candidates_out = widgets.Output()

state = GameState()
state.reset(N=N_widget.value, mode=mode_widget.value, k=k_widget.value, seed=seed_widget.value or None)

def refresh_ui():
    bin_range_start.max = state.N
    bin_range_end.max = state.N
    guess_input.max = state.N
    with status_out:
        clear_output()
        display(widgets.HTML(f"<h4>Numero segreto pronto. Candidati: {len(state.candidates)} • Tentativi: {state.attempts}</h4>"))
        if state.finished:
            display(widgets.HTML(f"<h4>✅ Trovato: <b>{state.secret}</b> in {state.attempts} tentativi.</h4>"))
    with history_out:
        clear_output()
        if state.history:
            for h in state.history:
                display(widgets.HTML(f"• {h}"))
        else:
            display(widgets.HTML("<i>Nessuna domanda ancora.</i>"))
    with candidates_out:
        clear_output()
        preview = state.candidates[:200]
        tail = "" if len(state.candidates) <= 200 else f" … (+{len(state.candidates)-200})"
        display(widgets.HTML(f"<b>Candidati ({len(state.candidates)}):</b> {preview}{tail}"))

def on_new_round_clicked(_):
    state.reset(N=N_widget.value, mode=mode_widget.value, k=k_widget.value, seed=seed_widget.value or None)
    refresh_ui()

def on_ask_bin_clicked(_):
    start = min(bin_range_start.value, bin_range_end.value)
    end = max(bin_range_start.value, bin_range_end.value)
    subset = list(range(start, end+1))
    state.answer_binary_subset(subset)
    refresh_ui()

def balanced_partitions(cands: List[int], k: int):
    parts = []
    n = len(cands)
    size = n // k
    r = n % k
    idx = 0
    for i in range(k):
        take = size + (1 if i < r else 0)
        parts.append(cands[idx: idx+take])
        idx += take
    return parts

def on_kary_partition_clicked(_):
    parts = balanced_partitions(state.candidates, k_widget.value)
    for i in range(10):
        if i < k_widget.value:
            k_parts_boxes[i].value = ",".join(map(str, parts[i]))
            k_parts_boxes[i].disabled = False
        else:
            k_parts_boxes[i].value = ""
            k_parts_boxes[i].disabled = True
    with kary_info_out:
        clear_output()
        display(widgets.HTML("Partizione bilanciata generata sulle candidate correnti."))

def on_kary_apply_clicked(_):
    parts = []
    used = set()
    for i in range(k_widget.value):
        txt = k_parts_boxes[i].value.strip()
        if txt == "":
            parts.append([]); continue
        try:
            nums = [int(x) for x in txt.replace(" ", "").split(",") if x != ""]
        except ValueError:
            nums = []
        clean = []
        for n in nums:
            if n in state.candidates and n not in used:
                clean.append(n); used.add(n)
        parts.append(clean)
    state.answer_kary_partition(parts)
    refresh_ui()

def on_guess_clicked(_):
    state.guess(guess_input.value)
    refresh_ui()

def on_record_clicked(_):
    record_result(state)
    df = pd.DataFrame(SESSION_RESULTS)
    with results_out:
        clear_output()
        if len(df):
            display(df[["mode","k","N","attempts","found","secret"]])
        else:
            display(widgets.HTML("<i>Nessun risultato ancora.</i>"))

# Bind
new_round_btn.on_click(on_new_round_clicked)
ask_bin_btn.on_click(on_ask_bin_clicked)
kary_partition_btn.on_click(on_kary_partition_clicked)
kary_apply_btn.on_click(on_kary_apply_clicked)
guess_btn.on_click(on_guess_clicked)
record_btn.on_click(on_record_clicked)

# Layout
top = widgets.HBox([N_widget, mode_widget, k_widget, seed_widget, new_round_btn])
bin_box = widgets.VBox([widgets.HTML("<h4>Domande binarie</h4>"), widgets.HBox([bin_range_start, bin_range_end, ask_bin_btn])])
kary_box = widgets.VBox([widgets.HTML("<h4>Domande k-arie</h4>"),
                         widgets.HBox([kary_partition_btn, kary_apply_btn]),
                         widgets.VBox(k_parts_boxes)])
guess_box = widgets.HBox([guess_input, guess_btn, record_btn])

ui = widgets.VBox([top, status_out,
                   widgets.HTML("<hr>"),
                   widgets.HBox([bin_box, kary_box]),
                   widgets.HTML("<hr>"),
                   widgets.HBox([widgets.VBox([widgets.HTML('<h4>Storia</h4>'), history_out]),
                                 widgets.VBox([widgets.HTML('<h4>Candidati</h4>'), candidates_out])]),
                   widgets.HTML("<hr>"),
                   widgets.VBox([widgets.HTML("<h4>Risultati sessione</h4>"), results_out]),
                   widgets.HTML("<hr>"),
                   guess_box])

display(ui)
refresh_ui()



## 4) Dopo aver giocato: bit richiesti (base k)
- Domande binarie: $\approx \log_2 N$ domande.
- Domande k-arie: $\approx \log_k N = \log_2 N / \log_2 k$ domande.
Il totale di **informazione** necessario dipende solo da **N**.


In [9]:

import pandas as pd, math
def theoretical_bits(N): return math.log(N, 2)
def theoretical_questions(N, k): return math.log(N, k)

if SESSION_RESULTS:
    df = pd.DataFrame(SESSION_RESULTS)
    agg = df.groupby(["mode","k","N"]).attempts.agg(["count","mean","min","max"]).reset_index()
    agg["theory_bits"] = agg["N"].apply(theoretical_bits)
    agg["theory_questions_base_k"] = agg.apply(lambda r: theoretical_questions(r["N"], r["k"]), axis=1)
    display(agg)
else:
    print("Gioca e registra qualche round, poi riesegui questa cella.")


,mode,k,N,count,mean,min,max,theory_bits,theory_questions_base_k
0,binary,2,64,1,3.0,3,3,6.0,6.0



## 5) **Entropia**: distribuzione **uniforme** vs **non uniforme**
Dividiamo l'intervallo in **B blocchi** (es. quartili) e assegniamo **pesi**; all'interno del blocco i numeri sono equiprobabili.
- **Uniforme** → $H = \log_2 N$ (massima incertezza).
- **Non uniforme** → $H < \log_2 N$ (meno informazione media).
Mostriamo anche una **prima domanda binaria** che separa ~metà della massa (greedy).


In [13]:

import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

N_entropy = widgets.BoundedIntText(value=64, min=4, max=4096, step=1, description="N (1..N):")
B_bins = widgets.BoundedIntText(value=4, min=2, max=16, step=1, description="Blocchi B:")
gen_sliders_btn = widgets.Button(description="↔️ Rigenera pesi")
norm_btn = widgets.Button(description="🔄 Normalizza pesi")
preset_uniform_btn = widgets.Button(description="🎯 Uniforme")
preset_skew_btn = widgets.Button(description="📈 Skew")
calc_btn = widgets.Button(description="📊 Calcola entropia e mostra", button_style="success")

sliders_box = widgets.VBox([])
out_entropy = widgets.Output()

def make_weight_sliders(B):
    return [widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.01, description=f"peso {i+1}", readout_format=".2f", continuous_update=False) for i in range(B)]

WEIGHT_SLIDERS = make_weight_sliders(B_bins.value)

def rebuild_sliders(*args):
    global WEIGHT_SLIDERS
    WEIGHT_SLIDERS = make_weight_sliders(B_bins.value)
    sliders_box.children = WEIGHT_SLIDERS

def normalize_weights(_=None):
    s = sum(sld.value for sld in WEIGHT_SLIDERS)
    if s <= 0:
        for sld in WEIGHT_SLIDERS: sld.value = 1.0
        s = sum(sld.value for sld in WEIGHT_SLIDERS)
    for sld in WEIGHT_SLIDERS:
        sld.value = sld.value / s * len(WEIGHT_SLIDERS)

def preset_uniform(_=None):
    for sld in WEIGHT_SLIDERS: sld.value = 1.0

def preset_skew(_=None):
    B = len(WEIGHT_SLIDERS)
    vals = np.linspace(2*B, B, B)
    vals = vals / vals.mean()
    for sld, v in zip(WEIGHT_SLIDERS, vals): sld.value = float(v)

def build_prior(N, B, weights):
    idxs = list(range(1, N+1))
    sizes = [N//B + (1 if i < (N % B) else 0) for i in range(B)]
    parts = []
    start = 0
    for sz in sizes:
        parts.append(idxs[start:start+sz])
        start += sz
    w = np.array(weights, dtype=float)
    w = np.maximum(w, 0.0)
    if w.sum() == 0: w = np.ones(B)
    w = w / w.sum()
    p = np.zeros(N, dtype=float)
    for bi, block in enumerate(parts):
        if len(block) == 0: continue
        p_block = w[bi] / len(block)
        for n in block:
            p[n-1] = p_block
    return p, parts, w

def entropy_bits(p):
    p = np.array(p, dtype=float)
    p = p[p>0]
    return float(-(p * np.log2(p)).sum())

def plot_prior(p):
    plt.figure()
    plt.bar(np.arange(1, len(p)+1), p)
    plt.xlabel("Numero"); plt.ylabel("Probabilità"); plt.title("Distribuzione a priori sui numeri")
    plt.show()

def calc_entropy_and_show(_=None):
    N = N_entropy.value
    B = B_bins.value
    weights = [sld.value for sld in WEIGHT_SLIDERS]
    p, parts, w_norm = build_prior(N, B, weights)
    H = entropy_bits(p)
    Hunif = math.log2(N)
    with out_entropy:
        clear_output()
        print(f"Entropia non uniforme H(P) = {H:.3f} bit")
        print(f"Entropia uniforme H(U) = log2(N) = {Hunif:.3f} bit")
        print(f"Δ = H(U) - H(P) = {Hunif - H:.3f} bit  → meno informazione media da scoprire.")
        cumsum = np.cumsum(p)
        split_idx = int(np.searchsorted(cumsum, 0.5)) + 1
        mass_left = cumsum[split_idx-1]
        mass_right = 1.0 - mass_left
        print(f"Prima domanda binaria (greedy): è ≤ {split_idx}?  (massa sinistra ≈ {mass_left:.3f}, destra ≈ {mass_right:.3f})")
        plot_prior(p)

gen_sliders_btn.on_click(rebuild_sliders)
norm_btn.on_click(normalize_weights)
preset_uniform_btn.on_click(preset_uniform)
preset_skew_btn.on_click(preset_skew)
calc_btn.on_click(calc_entropy_and_show)
B_bins.observe(lambda ch: rebuild_sliders(), names='value')

display(widgets.HBox([N_entropy, B_bins]))
display(widgets.HBox([gen_sliders_btn, norm_btn, preset_uniform_btn, preset_skew_btn]))
display(sliders_box)
rebuild_sliders()
display(calc_btn, out_entropy)


VBox()

Button(button_style='success', description='📊 Calcola entropia e mostra', style=ButtonStyle())

Output()

## 6) Information Gain

$$
IG(Y;X) = H(Y) - H(Y \mid X)
$$

In [11]:
import math
from collections import Counter

def entropy(values):
    """Compute Shannon entropy of a list of categorical values (base 2)."""
    counts = Counter(values)
    total = len(values)
    H = 0.0
    for c in counts.values():
        p = c/total
        H -= p * math.log2(p)
    return H

def information_gain(Y, X):
    """Compute IG(Y;X) for categorical Y and feature X."""
    H_Y = entropy(Y)
    # Conditional entropy
    H_Y_given_X = 0.0
    total = len(Y)
    for x_val in set(X):
        # indices where X == x_val
        indices = [i for i,v in enumerate(X) if v == x_val]
        subset_Y = [Y[i] for i in indices]
        H_sub = entropy(subset_Y)
        H_Y_given_X += (len(indices)/total) * H_sub
    IG = H_Y - H_Y_given_X
    return H_Y, H_Y_given_X, IG

# --- Esempio dataset (12 istanze)
Y = ["Sì"]*6 + ["No"]*6
# Una feature X che divide in due gruppi 5/1 e 1/5
X = ["A"]*6 + ["B"]*6
Y[:6] = ["Sì"]*5 + ["No"]*1
Y[6:] = ["Sì"]*1 + ["No"]*5

H_Y, H_Y_given_X, IG = information_gain(Y, X)
print(f"H(Y) = {H_Y:.3f} bit")
print(f"H(Y|X) = {H_Y_given_X:.3f} bit")
print(f"IG(Y;X) = {IG:.3f} bit")

H(Y) = 1.000 bit
H(Y|X) = 0.650 bit
IG(Y;X) = 0.350 bit


In [12]:
import math
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt

def H_binary(p1):
    """Entropia binaria H([p1, 1-p1]) in bit."""
    if p1 <= 0.0 or p1 >= 1.0:
        return 0.0
    return - (p1*math.log2(p1) + (1-p1)*math.log2(1-p1))

def safe_div(a, b):
    return a / b if b != 0 else 0.0

# Slider per conteggi
A_yes = widgets.IntSlider(value=5, min=0, max=100, step=1, description="A: Sì")
A_no  = widgets.IntSlider(value=1, min=0, max=100, step=1, description="A: No")
B_yes = widgets.IntSlider(value=1, min=0, max=100, step=1, description="B: Sì")
B_no  = widgets.IntSlider(value=5, min=0, max=100, step=1, description="B: No")

reset_btn = widgets.Button(description="Reset 5/1 e 1/5", button_style="warning")
random_btn = widgets.Button(description="Randomizza")
out = widgets.Output()

def update(_=None):
    with out:
        clear_output(wait=True)
        # Totali
        n_A = A_yes.value + A_no.value
        n_B = B_yes.value + B_no.value
        n_tot = n_A + n_B

        # Distribuzione a priori Y
        Y_yes = A_yes.value + B_yes.value
        Y_no  = A_no.value  + B_no.value
        p_yes = safe_div(Y_yes, n_tot)
        H_Y = H_binary(p_yes) if n_tot > 0 else 0.0

        # Entropie condizionate
        p_yes_A = safe_div(A_yes.value, n_A) if n_A > 0 else 0.0
        p_yes_B = safe_div(B_yes.value, n_B) if n_B > 0 else 0.0
        H_Y_A = H_binary(p_yes_A) if n_A > 0 else 0.0
        H_Y_B = H_binary(p_yes_B) if n_B > 0 else 0.0
        H_Y_given_X = safe_div(n_A, n_tot)*H_Y_A + safe_div(n_B, n_tot)*H_Y_B if n_tot > 0 else 0.0

        IG = H_Y - H_Y_given_X

        # Testo
        print(f"Totale: {n_tot}  |  A: {n_A}  B: {n_B}")
        print(f"Y:  Sì={Y_yes},  No={Y_no}")
        print(f"H(Y) = {H_Y:.3f} bit")
        print(f"H(Y|X) = {H_Y_given_X:.3f} bit  (A: {H_Y_A:.3f},  B: {H_Y_B:.3f})")
        print(f"IG(Y;X) = {IG:.3f} bit\n")

        # Grafico 1: distribuzione a priori
        plt.figure()
        plt.bar(["Sì","No"], [p_yes, 1-p_yes] if n_tot>0 else [0,0])
        plt.title("Distribuzione a priori Y (probabilità)")
        plt.ylim(0,1)
        plt.show()

        # Grafico 2: entropie per ramo
        plt.figure()
        labels = ["H(Y|A)", "H(Y|B)", "H(Y|X) media"]
        vals = [H_Y_A, H_Y_B, H_Y_given_X]
        plt.bar(labels, vals)
        plt.title("Entropie condizionate (bit)")
        plt.ylim(0, 1.0)
        plt.show()

        # Grafico 3: Information Gain
        plt.figure()
        plt.bar(["IG(Y;X)"], [IG])
        plt.title("Information Gain (bit)")
        plt.ylim(0, max(1.0, IG*1.2 + 0.05))
        plt.show()

def on_reset(_):
    A_yes.value, A_no.value = 5, 1
    B_yes.value, B_no.value = 1, 5
    update()

def on_random(_):
    import random
    A_yes.value = random.randint(0, 10)
    A_no.value  = random.randint(0, 10)
    B_yes.value = random.randint(0, 10)
    B_no.value  = random.randint(0, 10)
    if (A_yes.value + A_no.value + B_yes.value + B_no.value) == 0:
        A_yes.value = 1
    update()

# Eventi
A_yes.observe(update, 'value')
A_no.observe(update, 'value')
B_yes.observe(update, 'value')
B_no.observe(update, 'value')
reset_btn.on_click(on_reset)
random_btn.on_click(on_random)

display(widgets.VBox([
    widgets.HBox([A_yes, A_no]),
    widgets.HBox([B_yes, B_no]),
    widgets.HBox([reset_btn, random_btn]),
    out
]))

update()
